In [1]:
!pip install yfinance
!pip install ta
!pip install openpyxl
!pip install tqdm

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=4cf3e363eeb08109344599db3cbfc3d29eea45c1d0ec50d85e354f47d0945da6
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [2]:
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.trend import ADXIndicator
from ta.volatility import AverageTrueRange
from ta.trend import SMAIndicator
from ta.momentum import RSIIndicator
from ta.volume import VolumeWeightedAveragePrice
import math
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import openpyxl
from tqdm import tqdm


In [3]:
# Get the stock symbols from S&P 500
sp500 = pd.read_excel("/Users/zhongyitong/Library/Mobile Documents/com~apple~CloudDocs/UG_Study/AY_2023:2024/IS4226/Group Project/Complete-List-of-SP-500-Index-Constituents-Nov-1-2023.xlsx")
sp500_symbols = sp500['Ticker'].tolist()

FileNotFoundError: ignored

In [ ]:
start_date = '2016-01-01'
end_date = '2020-01-01'
interval = '1d'

# Get the stock data from Yahoo Finance
stock_data = pd.DataFrame(columns=['Symbol', 'Company', 'Industry', 'stock_df'])

for symbol in tqdm(sp500_symbols):
    company = sp500.loc[sp500['Ticker'] == symbol, 'Company Name'].iloc[0]
    industry = sp500.loc[sp500['Ticker'] == symbol, 'Sector'].iloc[0]
    hist_stock = yf.Ticker(symbol).history(start=start_date, end=end_date, interval=interval)
    bt_data = pd.DataFrame()
    bt_data["Close_Price"] = hist_stock["Close"]
    # Remove the symbols that have missing values
    if bt_data.isnull().values.any():
        continue
    else:
        bt_data["Return"] = np.log(bt_data["Close_Price"] / bt_data["Close_Price"].shift(1))
        stock_data = pd.concat([stock_data, pd.DataFrame({'Symbol': [symbol], 'Company': [company], 'Industry': [industry], 'stock_df': [bt_data]})])

# Reset index
stock_data = stock_data.reset_index(drop=True)

# Get the number of stocks
num_stocks = len(stock_data)

# Print the unique list of symbols
print(stock_data['Symbol'].unique())

100%|██████████| 505/505 [01:38<00:00,  5.11it/s]

['AAPL' 'MSFT' 'AMZN' 'NVDA' 'GOOGL' 'GOOG' 'META' 'BRKB' 'TSLA' 'UNH'
 'LLY' 'JPM' 'XOM' 'JNJ' 'V' 'PG' 'AVGO' 'MA' 'HD' 'CVX' 'MRK' 'ABBV'
 'PEP' 'COST' 'ADBE' 'KO' 'CSCO' 'WMT' 'TMO' 'MCD' 'PFE' 'CRM' 'BAC' 'ACN'
 'CMCSA' 'LIN' 'NFLX' 'ABT' 'ORCL' 'DHR' 'AMD' 'WFC' 'DIS' 'TXN' 'PM' 'VZ'
 'INTU' 'COP' 'CAT' 'AMGN' 'NEE' 'INTC' 'UNP' 'LOW' 'IBM' 'BMY' 'SPGI'
 'RTX' 'HON' 'BA' 'UPS' 'GE' 'QCOM' 'AMAT' 'NKE' 'PLD' 'NOW' 'BKNG' 'SBUX'
 'MS' 'ELV' 'MDT' 'GS' 'DE' 'ADP' 'LMT' 'TJX' 'T' 'BLK' 'ISRG' 'MDLZ'
 'GILD' 'MMC' 'AXP' 'SYK' 'REGN' 'VRTX' 'ETN' 'LRCX' 'ADI' 'SCHW' 'CVS'
 'ZTS' 'CI' 'CB' 'AMT' 'SLB' 'C' 'BDX' 'MO' 'PGR' 'TMUS' 'FI' 'SO' 'EOG'
 'BSX' 'CME' 'EQIX' 'MU' 'DUK' 'PANW' 'PYPL' 'AON' 'SNPS' 'ITW' 'KLAC'
 'ATVI' 'ICE' 'APD' 'SHW' 'CDNS' 'CSX' 'NOC' 'CL' 'MPC' 'HUM' 'FDX' 'WM'
 'MCK' 'TGT' 'ORLY' 'HCA' 'FCX' 'EMR' 'PXD' 'MMM' 'MCO' 'ROP' 'CMG' 'PSX'
 'MAR' 'PH' 'APH' 'GD' 'USB' 'NXPI' 'AJG' 'NSC' 'PNC' 'VLO' 'GBP' 'F'
 'MSI' 'GM' 'TT' 'EW' 'CARR' 'AZO' 'ADSK' 'TDG' 'ANET' 'SRE'

In [ ]:
# Print the length of the unique symbols
print(len(stock_data['Symbol'].unique()))

505


In [ ]:
print(stock_data['stock_df'][0])

                           Close_Price    Return
Date                                            
2016-01-04 00:00:00-05:00    23.977474       NaN
2016-01-05 00:00:00-05:00    23.376616 -0.025379
2016-01-06 00:00:00-05:00    22.919147 -0.019763
2016-01-07 00:00:00-05:00    21.951851 -0.043121
2016-01-08 00:00:00-05:00    22.067926  0.005274
...                                ...       ...
2019-12-24 00:00:00-05:00    69.236252  0.000950
2019-12-26 00:00:00-05:00    70.609901  0.019646
2019-12-27 00:00:00-05:00    70.583115 -0.000379
2019-12-30 00:00:00-05:00    71.002014  0.005917
2019-12-31 00:00:00-05:00    71.520813  0.007280

[1006 rows x 2 columns]


In [ ]:
print(stock_data[stock_data['Symbol'] == 'AAPL']['stock_df'].iloc[0])

                           Close_Price  Index_Close_Price    Return  \
Date                                                                  
2010-01-04 00:00:00-05:00     6.478998        1132.989990       NaN   
2010-01-05 00:00:00-05:00     6.490199        1136.520020  0.001727   
2010-01-06 00:00:00-05:00     6.386963        1137.140015 -0.016034   
2010-01-07 00:00:00-05:00     6.375158        1141.689941 -0.001850   
2010-01-08 00:00:00-05:00     6.417540        1144.979980  0.006626   
...                                ...                ...       ...   
2019-12-24 00:00:00-05:00    69.236244        3223.379883  0.000950   
2019-12-26 00:00:00-05:00    70.609909        3239.909912  0.019646   
2019-12-27 00:00:00-05:00    70.583107        3240.020020 -0.000380   
2019-12-30 00:00:00-05:00    71.002029        3221.290039  0.005918   
2019-12-31 00:00:00-05:00    71.520821        3230.780029  0.007280   

                           Index_Return  
Date                              

In [ ]:
first_date_low = "2016-01-04"
first_date_high = "2016-01-06"
last_date_low = "2019-12-27"
last_date_high = "2019-12-31"

# Convert string to datetime format
first_date_low = pd.to_datetime(first_date_low)
first_date_high = pd.to_datetime(first_date_high)
last_date_low = pd.to_datetime(last_date_low)
last_date_high = pd.to_datetime(last_date_high)

to_be_removed_symbols = []
# Loop through each stock
for index, row in tqdm(stock_data.iterrows()):
    symbol = row['Symbol']
    stock_df = row['stock_df']

    if stock_df.empty:
        to_be_removed_symbols.append(symbol)
    else:
        # Get the first date of the stock
        stock_first_date = stock_df.index[0]

        # Get the last date of the stock
        stock_last_date = stock_df.index[-1]

        # Convert tz-aware to tz-naive
        stock_first_date = stock_first_date.tz_localize(None)
        stock_last_date = stock_last_date.tz_localize(None)

        if (stock_first_date >= first_date_low and stock_first_date <= first_date_high) and (stock_last_date >= last_date_low and stock_last_date <= last_date_high):
            continue
        else:
            to_be_removed_symbols.append(symbol)

# Remove the symbols that do not have the full range of data
stock_data = stock_data[~stock_data['Symbol'].isin(to_be_removed_symbols)]

# Reset index
stock_data = stock_data.reset_index(drop=True)

# Get the number of stocks
num_stocks = len(stock_data)

print(num_stocks)



505it [00:00, 5580.35it/s]

484


In [ ]:
# Get the symbols of the stocks
available_symbols = stock_data['Symbol'].tolist()

list_of_sets = []
# Randomly select 10 different stocks for 1000 different sets randomly
for i in tqdm(range(1000)):
    random_state = np.random.RandomState(i)
    random_symbols = random_state.choice(available_symbols, size=10, replace=False)
    list_of_sets.append(list(random_symbols))

total_df_columns = ['index', 'df']
total_df = pd.DataFrame()
for i in tqdm(range(5)):
    column_names = ['index', 'list']
    new_df = pd.DataFrame(columns=column_names)
    for j in tqdm(range(0+i*200, 200+i*200)):
        random_symbols = list_of_sets[j]
        new_df = pd.concat([new_df, pd.DataFrame({'index': [j], 'list': [random_symbols]})])

    new_df = new_df.reset_index(drop=True)
    total_df = pd.concat([total_df, pd.DataFrame({'index': [i], 'df': [new_df]})])

total_df = total_df.reset_index(drop=True)

# For each of the dataframes in total_df, write it to an csv file
for index, row in tqdm(total_df.iterrows()):
    df = row['df']
    csv_name = 'df' + str(index) + '.csv'
    df.to_csv(csv_name, index=False)



100%|██████████| 5/5 [00:00<00:00,  7.49it/s]
5it [00:00, 169.95it/s]
